In [1]:
from local.node import Node
from local.constnode import ConstNode
from local.varnode import VarNode
from local.funcnode import FuncNode
from local.func import *
from local.treeoptimizer import TreeOptimizer
from local.crossover import *
from local.Optimizer import *

import os
import copy
import random
import pickle
import graphviz
import datetime
from PIL import Image
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, losses, metrics, optimizers

In [2]:

BATCH_SIZE = 512
EPOCHS = 10
def load_data():
    (x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
    x_train, x_test = x_train[..., tf.newaxis] / 255.0, x_test[..., tf.newaxis] / 255.0

    train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(BATCH_SIZE)
    test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(BATCH_SIZE)

    return train_dataset, test_dataset

def build_model():
    input = layers.Input(shape = (32, 32, 3))
    x = layers.Conv2D(32, 3, 2)(input)
    x = layers.LeakyReLU()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(units = 32, activation = "relu")(x)
    output = layers.Dense(units = 10, activation = "softmax")(x)

    return models.Model(input, output)

train_dataset, test_dataset = load_data()

model = build_model()
model.save_weights("model.keras")

In [3]:
# tensorflow
optimizer = optimizers.SGD(learning_rate = 0.001, momentum=0.9)

model.load_weights("model.keras")

loss_obj = metrics.Mean()
accuracy_obj = metrics.SparseCategoricalAccuracy()
val_loss_obj = metrics.SparseCategoricalCrossentropy()
val_accuracy_obj = metrics.SparseCategoricalAccuracy()

@tf.function
def train_step(X, Y):
    with tf.GradientTape() as tape:
        pred = model(X)
        loss = losses.SparseCategoricalCrossentropy()(Y, pred)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    loss_obj(loss)
    accuracy_obj(Y, pred)

@tf.function
def test_step(X, Y):
    pred = model(X)
    
    val_loss_obj(Y, pred)
    val_accuracy_obj(Y, pred)

for i in range(EPOCHS):
    
    loss_obj.reset_state()
    accuracy_obj.reset_state()
    val_loss_obj.reset_state()
    val_accuracy_obj.reset_state()

    for X, Y in tqdm(train_dataset):
        train_step(X, Y)
    
    for X, Y in tqdm(test_dataset):
        test_step(X, Y)

    print(f"epoch : {i}, loss : {float(loss_obj.result())}, accuracy : {float(accuracy_obj.result())}")
    print(f"val_loss : {float(val_loss_obj.result())}, val_accuracy : {float(val_accuracy_obj.result())}")

  0%|          | 0/98 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:00<00:00, 76.92it/s]


epoch : 0, loss : 2.2521066665649414, accuracy : 0.1774200052022934
val_loss : 2.1959075927734375, val_accuracy : 0.22059999406337738


100%|██████████| 20/20 [00:00<00:00, 185.19it/s]


epoch : 1, loss : 2.1407902240753174, accuracy : 0.2400600016117096
val_loss : 2.083075523376465, val_accuracy : 0.2547999918460846


...

epoch : 7, loss : 1.828418493270874, accuracy : 0.36410000920295715
val_loss : 1.819519281387329, val_accuracy : 0.36169999837875366


100%|██████████| 20/20 [00:00<00:00, 176.99it/s]


epoch : 8, loss : 1.812040090560913, accuracy : 0.37022000551223755
val_loss : 1.8037052154541016, val_accuracy : 0.3686000108718872


100%|██████████| 20/20 [00:00<00:00, 175.40it/s]

epoch : 9, loss : 1.7973476648330688, accuracy : 0.37619999051094055
val_loss : 1.789642333984375, val_accuracy : 0.37470000982284546


In [4]:
optimizer = SGD(learning_rate = 0.01)

model.load_weights("model.keras")

loss_obj = metrics.Mean()
accuracy_obj = metrics.SparseCategoricalAccuracy()
val_loss_obj = metrics.SparseCategoricalCrossentropy()
val_accuracy_obj = metrics.SparseCategoricalAccuracy()

# @tf.function
def train_step(X, Y):
    with tf.GradientTape() as tape:
        pred = model(X)
        loss = losses.SparseCategoricalCrossentropy()(Y, pred)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    loss_obj(loss)
    accuracy_obj(Y, pred)

@tf.function
def test_step(X, Y):
    pred = model(X)
    
    val_loss_obj(Y, pred)
    val_accuracy_obj(Y, pred)

for i in range(EPOCHS):
    
    loss_obj.reset_state()
    accuracy_obj.reset_state()
    val_loss_obj.reset_state()
    val_accuracy_obj.reset_state()

    for X, Y in tqdm(train_dataset):
        train_step(X, Y)
    
    for X, Y in tqdm(test_dataset):
        test_step(X, Y)

    print(f"epoch : {i}, loss : {float(loss_obj.result())}, accuracy : {float(accuracy_obj.result())}")
    print(f"val_loss : {float(val_loss_obj.result())}, val_accuracy : {float(val_accuracy_obj.result())}")

  0%|          | 0/98 [00:00<?, ?it/s]

100%|██████████| 20/20 [00:00<00:00, 92.17it/s]


epoch : 0, loss : 2.239865779876709, accuracy : 0.18702000379562378
val_loss : 2.1814911365509033, val_accuracy : 0.21870000660419464


100%|██████████| 20/20 [00:00<00:00, 176.99it/s]


epoch : 1, loss : 2.1212944984436035, accuracy : 0.2475000023841858
val_loss : 2.0659573078155518, val_accuracy : 0.27390000224113464


...

epoch : 7, loss : 1.8336005210876465, accuracy : 0.3599399924278259
val_loss : 1.819542407989502, val_accuracy : 0.3666999936103821


100%|██████████| 20/20 [00:00<00:00, 178.57it/s]


epoch : 8, loss : 1.8176518678665161, accuracy : 0.36601999402046204
val_loss : 1.803631067276001, val_accuracy : 0.37389999628067017


100%|██████████| 20/20 [00:00<00:00, 150.38it/s]

epoch : 9, loss : 1.803520679473877, accuracy : 0.3716199994087219
val_loss : 1.790061116218567, val_accuracy : 0.37779998779296875
